# Практическая работа №1 
## по предмету "Системы искусственного интеллекта"

В данной практической работе Вы будете работать с базой данных, посвященной баскетболу, которая включает в себя информацию об игроках, играх и командах. 


### Задание 1
Вам необходимо запустить в docker базу данных PostgreSQL и выгрузить туда все данные из файла nba.sqlite, приложенного к лабороторной работе.
Выгрузить данные можно с помощью утилиты pgloader (https://pgloader.readthedocs.io/en/latest/ref/sqlite.html)

### Задание 2

Загрузите данные из таблицы game. Посмотрите, какие есть типы игр в сезоне (season_type) и количество этих игр.

In [1]:
%%sql
select *
from game;


In [2]:
import pandas as pd

In [3]:
df_game = df_sql
df_game['season_type'].value_counts()

Нам понадобятся только регулярные игры сезона и play-off. Отделите эти игры в новый датафрейм.

In [4]:
df_regular_play_off = df_game[(df_game['season_type'] == 'Playoffs') | (df_game['season_type'] == 'Regular Season')]
df_regular_play_off['season_type'].value_counts()

В датах игр выделите только год, убрав день и месяц, в которых была проведена игра

In [5]:
df_regular_play_off['game_year'] = df_regular_play_off['game_date'].apply(lambda x: x.split('-')[0])
df_regular_play_off['game_year']

Нас интересует общий счет, поэтому уберите столбцы pts_home и pts_away, заменив их стобцом с их суммой

In [6]:
df_regular_play_off['pts_home_away'] = df_regular_play_off['pts_home'] + df_regular_play_off['pts_away']
df_regular_play_off['pts_home_away']

In [7]:
df_regular_play_off

Постройте график, где по оси Х будет год игр, а по оси Y - среднее количество очков за игру в этом году.

In [8]:
import seaborn as sns

sns.scatterplot(data=df_regular_play_off, x='game_year', y='pts_home_away', hue='season_type')

### Задание 3

Выполните запрос к таблице draft_history, получив датафрейм.

In [9]:
%%sql
SELECT * FROM draft_history;

In [10]:
df = df_sql1
df

Постройте график, где по оси X будет год, а по оси Y - количество выбранных игроков в этот год (каждая строка в таблице - выбранный игрок).

In [11]:
sns.countplot(data=df, x='season')

### Задание 4

Необходимо построить график, показывающий топ 10 школ, игроки из которых были выбраны. Для этого вам понадобятся таблицы common_player_info и draft_history.

In [12]:
%%sql
SELECT * FROM common_player_info;

In [13]:
%%sql
SELECT * FROM draft_history

In [14]:
%%sql
SELECT * 
FROM common_player_info AS C
INNER JOIN main.draft_history dh 
    ON C.person_id = dh.person_id

In [15]:
df_draft = df_sql4
df_draft['school'].value_counts().head(10)


In [16]:
schools = df_draft['school'].value_counts().head(10)
list(schools.index)

In [17]:
df_top_10_school = df_draft[df_draft['school'].isin(schools.index)]
df_top_10_school

In [18]:
sns.countplot(data=df_top_10_school, x='school')

### Задание 5

Постройте график сравнения выигранных домашних и выездных игр в каждом году. Для этого используйте таблицу game.

In [19]:
df_game = df_sql
df_game

In [20]:
df_game['game_year'] = df_game['game_date'].apply(lambda x: x.split('-')[0])
df_game

In [21]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15,7), dpi=300)

sns.scatterplot(data=df_game, x='game_year', y='pts_home', hue='pts_away', ax=axes[0])
sns.scatterplot(data=df_game, x='game_year', y='pts_away', hue='pts_home', ax=axes[1])


### Задание 6

Какова средняя продолжительность карьеры активного игрока NBA? Используйте таблицы common_player_info и player. Активный игрок - это тот, у которого в колонке is_active стоит 1.

In [22]:
%%sql
SELECT * FROM common_player_info;

In [23]:
%%sql
SELECT * FROM player

In [24]:
df_sql7[df_sql7['is_active'] == 1]

In [25]:
%%sql
SELECT * 
FROM common_player_info AS C
INNER JOIN main.player p 
    ON C.person_id = p.id

In [26]:
df_active = df_sql8[df_sql8['is_active'] == 1]
df_active['career'] = df_active[['from_year', 'to_year']].apply(lambda df: df['to_year'] - df['from_year'], axis=1)
df_active


In [27]:
df_active['career'].describe()

### Задание 7

Каково соотношение позиций, на которых играют игроки? Используйте таблицу common_player_info.

In [28]:
df_sql6

In [29]:
sns.countplot(data=df_sql6, x='position')